## Import Dependencies

In [53]:
import csv
from pathlib import Path
import pandas as pd
import glob
from datetime import datetime, timedelta
import numpy as np
# from sklearn import preprocessing
import matplotlib.pyplot as plt
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LinearRegression, RidgeCV
# from sklearn.metrics import mean_squared_error
import seaborn as sns
import pandas_profiling as pp
from settings import *
# from settings import DATA_DIR

In [54]:
download_dir = 'D:\Performance_All'
download_dir

'D:\\Performance_All'

In [55]:
DATA_DIR

'data'

In [56]:
outputFileName = 'FMPerfProcessedALL.csv'
outpath = "/".join([DATA_DIR,outputFileName])
outpath

'data/FMPerfProcessedALL.csv'

# Get the Single-Family Loan Performance Data Files from Fannie Mae

From the Fannie Mae [website:](https://loanperformancedata.fanniemae.com/lppub/index.html#Single-Family_Loan_Performance_Data_Files) we downloaded the Performance dataset (26GB zip file)

The [Performance file layout](https://loanperformancedata.fanniemae.com/lppub-docs/FNMA_SF_Loan_Performance_File_layout.pdf) was as follows:
![Performance File Layout](images/PerformanceFileLayout.jpg)

In [57]:
# Create mapping for the header of the files based on above file format
allheaderline = ['id','rptPeriod','sellerName','currIntRate','currUPB','loanAge','monMatur','adjMonMatur',\
              'maturDate','mSA','deliqStatus','modFlag','zeroBalCode','zeroBalDate','lastPdInstDate',\
              'forecloDate','dispDate','forecloCost','propRepCost','recovCosts','miscCost','holdTaxCost',\
              'saleProceed','credEnhProceed','repurchProceed','otherForecloProceed','nonIntUPB',\
              'prinForgivBal','repurchMakeWholeProceedFlg','forecloPrinWriteOffAmnt','servActivIndicator']
colchoices = [0,9,10,12,16]
headerline = ['id','mSA','deliqStatus','zeroBalCode','forcloDate']

In [59]:
# Loop through each file from the unzipped download file - which gave a set of 76 individual files

#Create an empty array to hold dataframes to later concatonate
li = []

#Loop through every file in the directory and append into array
for in_path in Path(download_dir).glob('Performance*.txt'):
    df = pd.read_csv(in_path, sep="|", index_col=None, header=None, usecols=colchoices)
    df.columns=headerline
    df = df.set_index('id')
    # Isolate the deliqStatus column to speed up aggregation calculations
    deliq = df[['deliqStatus']].copy()
    # deliqStatus has one value of 'X' amongst integer values and NaN values
    # First replace 'X' values with NaN
    deliq['deliqStatus'] = deliq['deliqStatus'].replace('X',np.nan).astype(float)
    # Determine the number of zero (0) values representing successful payments for a loan
    deliqGood = deliq[deliq['deliqStatus'] == 0].groupby('id').count()
    # Determine the number of values > 0 representing delinquent payments for a loan
    deliqBad = deliq[deliq['deliqStatus'] > 0].groupby('id').count()
    # Determine the max() of deliqStatus representing highest numberdelinquent payments in a row for a loan
    deliqMax = deliq[deliq['deliqStatus'] > 0].groupby('id').max()
#     # Select only rows that have a zeroBalCode with a value ## Commented out to include ALL records
#     zbc_df = df[df['zeroBalCode'].notnull()]
    df = df.drop(columns=['deliqStatus'])
    #Merge the zbc_df with the deliqGood
    zbc_merge = pd.merge(df, deliqGood, on=['id'])
    #Merge the zbc_df with the deliqBad
    zbc_merge = pd.merge(zbc_merge, deliqBad, on=['id'])
    #Merge the zbc_df with the deliqMax
    zbc_merge = pd.merge(zbc_merge, deliqMax, on=['id'])
    # Rename all the deqliq* columns
    zbc_merge = zbc_merge.rename(columns={'deliqStatus_x':'deliqGood','deliqStatus_y':'deliqBad','deliqStatus':'deliqMax'})
    # Add the zbc_merge df to the li list to be able to concat them later
    li.append(zbc_merge)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\ProgramDa

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\ProgramDa

In [60]:
len(li)

78

In [61]:
#Concatonate all the array elements into one giant dataframe
processed_df = pd.concat(li, axis=0)
#df.columns = allheaderline

In [62]:
processed_df.shape

(443356941, 6)

In [63]:
processed_df.head()

,mSA,zeroBalCode,forcloDate,deliqGood,deliqBad,deliqMax
id,,,,,,
100021703104,35980,NaN,NaN,33,1,1.0
100021703104,35980,NaN,NaN,33,1,1.0
100021703104,35980,NaN,NaN,33,1,1.0
100021703104,35980,NaN,NaN,33,1,1.0
100021703104,35980,NaN,NaN,33,1,1.0


In [64]:
#Review the columns for datatypes and how many non-null values 
processed_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 443356941 entries, 100021703104 to 999888644871
Data columns (total 6 columns):
 #   Column       Dtype  
---  ------       -----  
 0   mSA          int64  
 1   zeroBalCode  float64
 2   forcloDate   object 
 3   deliqGood    int64  
 4   deliqBad     int64  
 5   deliqMax     float64
dtypes: float64(2), int64(3), object(1)
memory usage: 23.1+ GB


### Convert the origDate,firstPmtDate columns from strings to datetime datatypes
Since this will be such a large dataset I tested a number of methods to change datatypes

In [65]:
processed_df.shape

(443356941, 6)

In [66]:
processed_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 443356941 entries, 100021703104 to 999888644871
Data columns (total 6 columns):
 #   Column       Dtype  
---  ------       -----  
 0   mSA          int64  
 1   zeroBalCode  float64
 2   forcloDate   object 
 3   deliqGood    int64  
 4   deliqBad     int64  
 5   deliqMax     float64
dtypes: float64(2), int64(3), object(1)
memory usage: 23.1+ GB


In [73]:
processed_df.reset_index(inplace=True)


In [75]:
df = processed_df.copy()

In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 443356941 entries, 0 to 443356940
Data columns (total 7 columns):
 #   Column       Dtype  
---  ------       -----  
 0   id           int64  
 1   mSA          int64  
 2   zeroBalCode  float64
 3   forcloDate   object 
 4   deliqGood    int64  
 5   deliqBad     int64  
 6   deliqMax     float64
dtypes: float64(2), int64(4), object(1)
memory usage: 23.1+ GB


In [78]:
df.head(50)

,id,mSA,zeroBalCode,forcloDate,deliqGood,deliqBad,deliqMax
0,100021703104,35980,NaN,NaN,33,1,1.0
1,100021703104,35980,NaN,NaN,33,1,1.0
2,100021703104,35980,NaN,NaN,33,1,1.0
3,100021703104,35980,NaN,NaN,33,1,1.0
4,100021703104,35980,NaN,NaN,33,1,1.0
5,100021703104,35980,NaN,NaN,33,1,1.0
6,100021703104,35980,NaN,NaN,33,1,1.0
7,100021703104,35980,NaN,NaN,33,1,1.0
8,100021703104,35980,NaN,NaN,33,1,1.0
9,100021703104,35980,NaN,NaN,33,1,1.0


In [1]:
df.id.value_counts()

NameError: name 'df' is not defined

In [79]:
df.drop_duplicates( subset='id',keep='last', inplace=True)

MemoryError: Unable to allocate 13.2 GiB for an array with shape (4, 443356941) and data type int64

In [38]:
processed_df['zeroBalCode'].fillna(0, inplace=True)

In [39]:
processed_df['forcloDate'].fillna(0, inplace=True)

In [40]:
processed_df.loc[processed_df['forcloDate'] != 0, 'forcloDate'] =1

In [44]:
processed_df['zeroBalCode'] = processed_df['zeroBalCode'].astype(int)

In [45]:
processed_df['forcloDate'] = processed_df['forcloDate'].astype(int)

In [47]:
processed_df['deliqMax'] = processed_df['deliqMax'].astype(int)

In [48]:
processed_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 443356941 entries, 100021703104 to 999888644871
Data columns (total 6 columns):
 #   Column       Dtype
---  ------       -----
 0   mSA          int64
 1   zeroBalCode  int32
 2   forcloDate   int32
 3   deliqGood    int64
 4   deliqBad     int64
 5   deliqMax     int32
dtypes: int32(3), int64(3)
memory usage: 18.2 GB


In [42]:
#problem = processed_df[processed_df['forcloDate'] == 0 & processed_df['zeroBalCode'] != 0]

# Write the result df to csv

In [50]:
processed_df.to_csv(outpath)

In [51]:
processed_df.shape

(3594711, 6)

In [52]:
processed_df.head()

,mSA,zeroBalCode,forcloDate,deliqGood,deliqBad,deliqMax
id,,,,,,
100023274028,12060,0,0,20,11,4
100023274028,12060,1,0,20,11,4
100057706656,17140,6,0,13,1,1
100118647578,38900,0,0,35,21,2
100118647578,38900,1,0,35,21,2
